<a href="https://colab.research.google.com/github/ronaldoaf/999.079_AT_MUDA_STATE_JOB/blob/master/Atualiza_Regress%C3%A3o_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://bot-ao.com/half/weka_goaline.php

--2022-02-24 13:16:13--  http://bot-ao.com/half/weka_goaline.php
Resolving bot-ao.com (bot-ao.com)... 153.92.6.122, 2a02:4780:a:234:0:18be:ff35:3
Connecting to bot-ao.com (bot-ao.com)|153.92.6.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘weka_goaline.php’

weka_goaline.php        [             <=>    ]  54.21M  1.07MB/s    in 44s     

2022-02-24 13:16:59 (1.23 MB/s) - ‘weka_goaline.php’ saved [56848520]



In [ ]:
1.5 % 1

0.5

In [ ]:
import torch
from torch import log,from_numpy,no_grad,nn,optim, sign, abs as abs1
from torch.nn.functional import leaky_relu

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

from random import random

#Se x < min_ retorna 0, senão x
def re(x,min_):
    return x*(x>min_).float()


#df=pd.read_csv('weka_scale.csv')
df=pd.read_csv('weka_goaline.php')
df=df[df.goal_diff % 1==0.5]

df['X']=df.s_g/np.log(df.s_s+0.75)
df['y']=df.s_s**1.5
df['J1']=np.log(1+df.d_g)
df['K1']=np.log(1+df.s_g)
df['K2']=np.log(1+df.K1)
df['L1']=np.log(1+df.s_s)
df['L2']=np.log(1+df.L1)
df['L3']=np.log(1+df.L2)
df['M1']=np.log(1+df.goal_diff)
df['hand']=abs(df.handicap)
df['N1']=np.log(1+df.oddsU)


colunas='s_g,s_c,s_s,s_da,d_g,d_da,goal_diff,oddsU,K1,L1,W,hand'.split(',')
df=df[colunas+['pl_u']]


escala=MinMaxScaler().fit(df[colunas])

df[colunas]=escala.transform(df[colunas])

TT=0.5
df_train, df_test=train_test_split(df,  test_size=10000,shuffle=False)


#df_train= df_train[(df_train.goal_diff<=2/.7) &  (df_train.goal_diff>=(1.25-0.25)/6.25 ) & (df_train.goal_diff<=(4.25-0.25)/6.25 )]
#df_test=df_test[(df_test.goal_diff<=2/.7) &   (df_test.goal_diff>=(1.25-0.25)/6.25 ) &  ( df_test.goal_diff<=(4.25-0.25)/6.25 )]

'''
tam=len(df_train)
a=df_train[:int(tam/2)]
b=df_train[int(tam/2):]
df_train=pd.concat([a,b,b])
'''
#df_train=df
#df_train=df_train[-450000:]

x=df_train.iloc[:,:-1]
y=df_train.iloc[:,-1:]

reg=LinearRegression().fit(x, y)




X=from_numpy(x.values).float()
Y=from_numpy(y.values).float()

modelo=nn.Sequential(
    nn.Linear(X.shape[1], 1),
    nn.LeakyReLU(negative_slope=TT)
)

#Otimiza para que o pred_y fique próximo do PL observado
loss_func = torch.nn.MSELoss()

otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.5)
                    

for _ in range(100):
  pred_y=modelo(X)
  loss=loss_func(pred_y, Y)


  otimizador.zero_grad() 
  loss.backward() 
  otimizador.step()




#Otimiza para maximizar o crescimento da banca    
def loss_somalog(y_pred,y):
    #return -log(1+y*y_pred.clamp(min=0, max=0.3)).sum()
    return -log(1+y*y_pred).sum()
    #return -log(1+y*leaky_relu(y_pred, negative_slope=-0.05 )).sum()



otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.1) 
for _ in range(2000):
    pred_y=modelo(X)
    loss=loss_somalog(pred_y,Y)
    otimizador.zero_grad() 
    loss.backward() 
    otimizador.step()


#with no_grad():
#    print(loss.item())


      
#Realiza o teste comparando a lucrativida da regressão linear com a rede neural    
Y=df_test.iloc[:,-1].values   
Y_nn=modelo(from_numpy(df_test.iloc[:,:-1].values).float()).cpu().detach().numpy()
Y_reg=reg.predict(df_test.iloc[:,:-1].values)


print('NN:', sum(np.log(1+y*y_pred) for y_pred,y in zip(Y_nn,Y) if y_pred>0.015 ) )

print('LR:', sum(np.log(1+y*y_pred*1.5) for y_pred,y in zip(Y_reg,Y) if y_pred>0.015) )

parms=[parm.data.numpy() for parm in modelo.parameters()][0][0]
inter=[parm.data.numpy() for parm in modelo.parameters()][1][0]

for p,c in zip(parms/(escala.data_max_-escala.data_min_), colunas):
    print(p,'*',c,'+')

print(sum(-parms*escala.data_min_/(escala.data_max_-escala.data_min_))+inter)

print(len(Y_reg),  len(Y_nn) )

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


NN: [3.3977325]
LR: [2.88539148]
0.03919266164302826 * s_g +
-0.007558978029659816 * s_c +
-0.0180872197334583 * s_s +
-0.0005782835651189089 * s_da +
-0.042195567062922885 * d_g +
-0.00019307710902124155 * d_da +
0.14966127872467042 * goal_diff +
0.23762169811460704 * oddsU +
-0.11828876305861284 * K1 +
0.04374330856472631 * L1 +
0.07334329932928085 * W +
-0.027245215007237027 * hand +
-0.5204384098812522
10000 10000


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
print('LR:', sum(np.log(1+y*y_pred)  for y_pred,y in zip(Y_reg,Y) if y_pred>0.015) )

LR: [2.82805767]


In [ ]:
print('NN:', sum(np.log(1+y*y_pred) for y_pred,y in zip(Y_nn,Y) if y_pred>0.015 ) )

NN: [3.3984811]


In [ ]:
df2 = pd.DataFrame()
df2['ev']=Y_reg.flatten()
df2['ke']=Y_nn.flatten()

df2[df2.ev>0].to_csv('df2.csv')

df2


,ev,ke
0,-0.033725,-0.047970
1,-0.054646,-0.081067
2,-0.042283,-0.070144
3,-0.030756,-0.048717
4,-0.007958,-0.022862
...,...,...
47674,-0.031398,-0.047215
47675,-0.087213,-0.107761
47676,-0.043904,-0.058559
47677,-0.105547,-0.130695


In [ ]:
df=pd.read_csv('weka_goaline.php')

df

,id,data_inicio,home,away,s_g,s_c,s_da,s_s,d_g,d_c,d_da,d_s,s_r,goal_diff,oddsU,oddsO,W,handicap,pl_u
0,58098431,2016-09-01 00:00:00,Ipatinga Women,Villa Nova Women,0,2,55,16,0,2,5,4,0,1.75,1.800,2.000,1,0.50,-0.500
1,58106036,2016-09-01 00:00:00,Botafogo PB Women,Nautico Capibaribe Women,2,4,28,11,0,0,6,5,0,1.50,1.900,1.900,1,-0.25,-1.000
2,58114652,2016-09-01 00:00:00,Charlotte Independence,FC Cincinnati,3,9,52,6,1,5,2,0,0,1.25,1.975,1.825,0,-0.25,-1.000
3,58114654,2016-09-01 00:00:00,FC Montreal,Wilmington Hammerheads,1,8,34,9,1,8,0,3,0,1.50,1.750,2.050,0,-0.25,-1.000
4,58114656,2016-09-01 00:00:00,Richmond Kickers,Charleston Battery,0,2,66,6,0,2,10,4,0,1.25,1.950,1.850,0,-0.25,-1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499253,105295473,2021-07-17 15:00:00,Queens Park,Airdrieonians,0,8,63,13,0,4,1,3,0,1.50,1.800,2.050,0,-0.25,0.800
499254,105295475,2021-07-17 15:00:00,Stranraer,East Kilbride,1,6,39,12,1,4,5,2,0,1.50,1.800,2.050,0,-0.25,0.800
499255,105295477,2021-07-17 15:00:00,Stenhousemuir,Partick,1,8,47,13,1,6,17,1,0,2.25,1.750,2.050,0,0.50,0.375
499256,105314861,2021-07-17 15:00:00,Crewe,Wolverhampton,0,4,37,6,0,0,3,2,0,1.50,1.825,1.975,0,0.25,0.825


In [ ]:
for c,m,M in zip(colunas,escala.data_min_,escala.data_max_):
  print([c,m,M])

['s_g', 0.0, 8.0]
['s_c', 0.0, 28.0]
['s_s', 0.0, 78.0]
['s_da', 1.0, 302.0]
['d_g', 0.0, 7.0]
['d_da', 0.0, 134.0]
['goal_diff', 0.25, 6.25]
['oddsU', 1.17, 5.0]
['K1', 0.0, 2.1972245773362196]
['L1', 0.0, 4.3694478524670215]
['W', 0.0, 1.0]
['hand', 0.0, 5.75]


In [ ]:
#@title Texto de título padrão

with no_grad():
    X2=torch.stack( [x for x, y_pred in zip(X,modelo(X)) if y_pred>0 ] )


X2

tensor([[0.2500, 0.1429, 0.0897,  ..., 0.5687, 1.0000, 0.0435],
        [0.0000, 0.0714, 0.2159,  ..., 0.4453, 0.0000, 0.0435],
        [0.2500, 0.0357, 0.1894,  ..., 0.4101, 0.0000, 0.0435],
        ...,
        [0.1250, 0.0714, 0.1628,  ..., 0.4759, 0.0000, 0.1739],
        [0.0000, 0.0714, 0.2990,  ..., 0.4101, 0.0000, 0.0870],
        [0.1250, 0.1786, 0.1196,  ..., 0.4101, 0.0000, 0.0435]])

In [ ]:
#df=pd.read_csv('weka_scale.csv')
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/weka.csv')
df['X']=df.s_g/np.log(df.s_s+0.75)
df['y']=df.s_s**1.5
df['L1']=np.log(1+df.s_s)
df['L2']=np.log(1+df.L1)
df['L3']=np.log(1+df.L2)
df['M1']=np.log(1+df.goal_diff)
df['hand']=abs(df.handicap)
df['N1']=np.log(1+df.oddsU)


colunas='s_g,s_c,s_da,s_s,d_g,d_da,goal_diff,oddsU,X,L1,W,hand'.split(',')
df=df[colunas+['pl_u']]


escala=MinMaxScaler().fit(df[colunas])

df[colunas]=escala.transform(df[colunas])



ROIS=[]
for i in range(1):

    #Embaralha o dataframe, apartir de um estado predefindo
    df=df.sample(frac=1.0, random_state=i)
    df_train=df[:-100000] 
    df_test=df[-100000:]
    
    df_train=df_train[(df_train.d_g<=5/7.) & (df_train.goal_diff>=(1.0-0.25)/5.5 ) & (df_train.goal_diff<=(4.5-0.25)/5.5 )]
    df_test=df_test[(df_test.d_g<=5/7.) & (df_test.goal_diff>=(1.0-0.25)/5.5 ) &  ( df_test.goal_diff<=(4.5-0.25)/5.5 )]

    x=df_train.iloc[:,:-1].values
    y=df_train.iloc[:,-1:].values


    X=from_numpy(x).float()
    Y=from_numpy(y).float()

    modelo=nn.Sequential(
        nn.Linear(X.shape[1], 1),
        nn.LeakyReLU(negative_slope=0.5)
    )


    #Otimiza para que o pred_y fique próximo do PL observado
    loss_func = torch.nn.MSELoss()
    #loss_func = torch.nn.SmoothL1Loss()
    otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.5) 

    for _ in range(1000):
        pred_y=modelo(X)

        loss = loss_func(pred_y, Y) 

        otimizador.zero_grad() 
        loss.backward() 
        otimizador.step()

        
        
        
    #Otimiza para maximizar o crescimento da banca    
    def loss_somalog(y_pred,y):
        #return -log(1+y*y_pred.relu().sum()
        return -log(1+y*y_pred).sum()
   
    otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.1) 
    for _ in range(1000):
        pred_y=modelo(X)

        loss = loss_somalog(pred_y, Y) 

        otimizador.zero_grad() 
        loss.backward() 
        otimizador.step()    
        
        
    #Realiza o teste comparando a lucrativida da regressão linear com a rede neural    
    Y=df_test.iloc[:,-1].values   
    Y_nn=modelo(from_numpy(df_test.iloc[:,:-1].values).float()).cpu().detach().numpy()


    roi=sum(np.log(1+y*y_pred ) for y_pred,y in zip(Y_nn,Y) if y_pred>0.00)

    ROIS+=[roi]

print(np.mean(ROIS), colunas)


43.67503 ['s_g', 's_c', 's_da', 's_s', 'd_g', 'd_da', 'goal_diff', 'oddsU', 'X', 'L1', 'W', 'hand']


In [ ]:


51.78604 0.33 

53.270897 0.4

54.427002  0.5

54.222485 0.66



In [ ]:
#df=pd.read_csv('weka_scale.csv')
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/weka.csv')
df['X']=df.s_g/np.log(df.s_s+0.75)
df['y']=df.s_s**1.5
df['L1']=np.log(1+df.s_s)
df['L2']=np.log(1+df.L1)
df['L3']=np.log(1+df.L2)
df['M1']=np.log(1+df.goal_diff)
df['hand']=abs(df.handicap)
df['N1']=np.log(1+df.oddsU)


colunas='s_g,s_c,s_da,s_s,d_g,d_da,goal_diff,oddsU,X,L1,W,hand'.split(',')
df=df[colunas+['pl_u']]


escala=MinMaxScaler().fit(df)

df[colunas+['pl_u']]=escala.transform(df)


def N(valor_trans):
  return valor_trans*(escala.data_max_[-1]-escala.data_min_[-1])+escala.data_min_[-1]

ROIS=[]
for i in range(1):

    #Embaralha o dataframe, apartir de um estado predefindo
    df=df.sample(frac=1.0, random_state=i)
    df_train=df[:-100000] 
    df_test=df[-100000:]
    
    df_train=df_train[(df_train.d_g<=5/7.) & (df_train.goal_diff>=(1.0-0.25)/5.5 ) & (df_train.goal_diff<=(4.5-0.25)/5.5 )]
    df_test=df_test[(df_test.d_g<=5/7.) & (df_test.goal_diff>=(1.0-0.25)/5.5 ) &  ( df_test.goal_diff<=(4.5-0.25)/5.5 )]

    x=df_train.iloc[:,:-1].values
    y=df_train.iloc[:,-1:].values


    X=from_numpy(x).float()
    Y=from_numpy(y).float()

    modelo=nn.Sequential(
        nn.Linear(X.shape[1], 1 ),
        nn.Sigmoid(),
    )


    #Otimiza para que o pred_y fique próximo do PL observado
    loss_func = torch.nn.MSELoss()
    #loss_func = torch.nn.SmoothL1Loss()
    otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.2) 

    for _ in range(1000):
        pred_y=modelo(X)

        loss = loss_func(pred_y, Y) 

        otimizador.zero_grad() 
        loss.backward() 
        otimizador.step()

        
        
        
    #Otimiza para maximizar o crescimento da banca    
    def loss_somalog(y_pred,y):
        #return -log(1+y*y_pred.relu().sum()
        return -log(1+N(y)*N(y_pred)).sum()


    otimizador = torch.optim.Adam(modelo.parameters(), lr = 0.1) 
    for _ in range(1000):
        pred_y=modelo(X)

        loss = loss_somalog(pred_y, Y) 

        otimizador.zero_grad() 
        loss.backward() 
        otimizador.step()    
        
    #Realiza o teste comparando a lucrativida da regressão linear com a rede neural    
    Y=N(df_test.iloc[:,-1].values)   
    Y_nn=N(modelo(from_numpy(df_test.iloc[:,:-1].values).float()).cpu().detach().numpy())

    roi=sum(np.log(1+y*y_pred ) for y_pred,y in zip(Y_nn,Y) if y_pred>0.00)

    ROIS+=[roi]

print(np.mean(ROIS), colunas)


0.0 ['s_g', 's_c', 's_da', 's_s', 'd_g', 'd_da', 'goal_diff', 'oddsU', 'X', 'L1', 'W', 'hand']


In [ ]:
nn.Le

array([-1.   ,  0.875,  0.975, ...,  0.925,  1.025,  0.95 ])